In [4]:
from newsplease import NewsPlease # An integrated web crawler and information extractor for news 
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys #  System-specific parameters and functions

In [ ]:
# for collecting the total number of article
article_count = int(''.join(i for i in soup.select_one('.section-controls').span.text.split('of')[1] if i.isdigit()))

# to find the no.of pages
max_pages = int(article_count)//12

In [2]:
%%time
Title, Link = [], []
for index, i in enumerate(range(max_pages)):    
    url = 'https://www.thehindu.com/search/?q=Alcoholics%20Anonymous&order=DESC&sort=publishdate&page=' + str(i)
    response = get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    # Extracts url's from Hindu about AA
    url = [soup.select('a.story-card75x1-text')[i]['href'] for i in range(len(soup.select('a.story-card75x1-text')))]
    Link.extend(url)
    
    # Extracts Headline's from Hindu about AA
    title = [soup.select('a.story-card75x1-text')[i].text.strip() for i in range(len(soup.select('a.story-card75x1-text')))]
    Title.extend(title)
    
    sys.stdout.write('\r' + str(index))
    sys.stdout.flush()

982Wall time: 18min 8s


In [8]:
%%time
headline, time, meeting, news, image, authors = [], [], [], [], [], []

for index, url in enumerate(Link):
    
    # Parse the url to NewsPlease 
    article = NewsPlease.from_url(url)
    
    # Extracts the Headlines of News Article related to AA
    try:
        headline.append(article.title)
    except:
        headline.append(None)
    
    # Extracts the Authors of News Article related to AA
    try:
        authors.append(article.authors)
    except:
        authors.append(None)
    
    # Extracts the Main_Image_url of News Article related to AA
    try:
        image.append(article.image_url)
    except:
        image.append(None)
    
    # Extracts the Published_Date of News Article related to AA
    try:
        time.append(str(article.date_publish))
    except: 
        time.append(None)
    
    # Extracts the AA meetings if available
    try:
        meeting.append(article.text.split('Alcoholics Anonymous:')[-1].strip())
    except:
        meeting.append(None)
      
    # Extracts the Main_News from Article related to AA
    try:
        news.append(article.text.strip())
    except:
        news.append(None)
        
    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 5h 46min 51shindu.com/todays-paper/tp-miscellaneous/in-madurai-today/article3235723.ece3236743.ece46450.ece9.ecece/article3169273.ece7.ece569.eceeceps-have-turned-life-around-for-many-troubled-young-people/article8509546.ece


In [9]:
tbl = pd.DataFrame({'Headline' : headline, 
                    'Authors' : authors, 
                    'Main_Image' : image, 
                    'Published_Date' : time, 
                    'Meetings' : meeting, 
                    'News' : news,
                    'Source_urls' : Link})
tbl

,Headline,Authors,Main_Image,Published_Date,Meetings,News,Source_urls
0,"Engagements for Sunday, March 31",[To Run A.M. Until P.M.],https://www.thehindu.com/static/theme/default/...,2019-03-31 01:07:44,"Meetings, Assemption School, St. Teresa Church...","RELIGION\nRamayanam: Mathivannan, Asthika Sama...",https://www.thehindu.com/news/cities/chennai/e...
1,Chennai Today,[],https://www.thehindu.com/static/theme/default/...,2019-03-30 00:00:00,"Meetings, Community Service Centre, Balfour Rd...","Religion\nKamba Ramayanam: Mathivannan, Asthik...",https://www.thehindu.com/todays-paper/tp-natio...
2,Madurai Today,[],https://www.thehindu.com/static/theme/default/...,2019-03-30 00:00:00,"Meeting; Ahana Hospital, Anna Bus Stand; Quali...",RELIGION\nMeenakshi Sundareswarar Temple: Thev...,https://www.thehindu.com/todays-paper/tp-natio...
3,engagements,[],https://www.thehindu.com/static/theme/default/...,2019-03-30 00:00:00,TIRUNELVELI\nAlcoholics Anonymous and Al-Anon:...,TIRUNELVELI\nAlcoholics Anonymous and Al-Anon:...,https://www.thehindu.com/todays-paper/tp-natio...
4,CHENNAI TODAY,[To Run A.M. Until P.M.],https://www.thehindu.com/static/theme/default/...,2019-03-29 01:44:55,"Meetings, Kevin School, Royapuram; Singaram Pi...",RELIGION\nRamayanam: Harikatha by Kalyanapuram...,https://www.thehindu.com/news/cities/chennai/c...
5,engagements,[],https://www.thehindu.com/static/theme/default/...,2019-03-29 00:00:00,TIRUNELVELI\nSri Jayendra Saraswathi Swamigal ...,TIRUNELVELI\nSri Jayendra Saraswathi Swamigal ...,https://www.thehindu.com/todays-paper/tp-natio...
6,"Madurai Today-March 29, 2019",[],https://www.thehindu.com/static/theme/default/...,2019-03-29 00:00:00,"Meeting, Cathedral Church, Narimedu; Divine Gr...",RELIGION\nTiruvalluvar Mandram: Discourse on ‘...,https://www.thehindu.com/todays-paper/tp-natio...
7,"Madurai Today-March 29, 2019","[Flying Squad Teams, Fsts, Static Surveillance...",https://www.thehindu.com/static/theme/default/...,2019-03-28 21:23:59,"Meeting, Cathedral Church, Narimedu; Divine Gr...",RELIGION\nTiruvalluvar Mandram: Discourse on ‘...,https://www.thehindu.com/news/cities/Madurai/m...
8,CHENNAI TODAY,[To Run A.M. Until P.M.],https://www.thehindu.com/static/theme/default/...,2019-03-28 02:28:35,"Meetings, Church of Christ, Anna Nagar; Police...",RELIGION\nRamayanam: Harikatha by Kalyanapuram...,https://www.thehindu.com/news/cities/chennai/c...
9,engagements,[],https://www.thehindu.com/static/theme/default/...,2019-03-28 00:00:00,TIRUNELVELI\nSadakkathullah Appa College: Vale...,TIRUNELVELI\nSadakkathullah Appa College: Vale...,https://www.thehindu.com/todays-paper/tp-natio...


In [13]:
tbl.to_excel(input("Filename:  //include '.xls' at the end of the filename"))

Filename:  //include '.xls' at the end of the filenameTHE_HINDU_AA_NEWS_DETAILS.xls


In [14]:
tbl.to_csv('THE_HINDU_AA_NEWS_DETAILS.csv', index=False)